#Data Collection Using Quantrocket

In [ ]:
from quantrocket.license import set_license
set_license('f116ff0d-ddf0-11ee-b05b-8b9fa8f82e74')

{'licensekey': 'f116........2e74', 'software_license': {}}

In [ ]:
from quantrocket.master import collect_usstock_listings
collect_usstock_listings()

{'status': 'success', 'msg': 'successfully loaded FREE stock listings'}

In [ ]:
from quantrocket.master import get_securities
securities = get_securities(symbols='AAPL', fields=['Sid','Symbol','Exchange'], vendors='usstock')
securities

,Symbol,Exchange
Sid,,
FIBBG000B9XRY4,AAPL,XNAS


In [ ]:
aapl = securities.index[0]
aapl #aapl security id

'FIBBG000B9XRY4'

In [ ]:
from quantrocket.zipline import create_usstock_bundle
create_usstock_bundle("usstock-free-1min", free=True)

HTTPError: ('400 Client Error: BAD REQUEST for url: http://houston/zipline/bundles/usstock-free-1min?ingest_type=usstock&free=True', {'status': 'error', 'msg': 'bundle usstock-free-1min already exists'})

In [ ]:
from quantrocket.zipline import ingest_bundle
ingest_bundle("usstock-free-1min")

{'status': 'the data will be ingested asynchronously'}

In [ ]:
from quantrocket import get_prices
data = get_prices("usstock-free-1min", data_frequency='daily', sids=aapl, start_date='2023-01-01', end_date='2023-12-31', fields="Close")

In [ ]:
data

Sid               FIBBG000B9XRY4
Field Date                      
Close 2023-01-03         124.375
      2023-01-04         125.658
      2023-01-05         124.325
      2023-01-06         128.900
      2023-01-09         129.427
...                          ...
      2023-12-22         193.600
      2023-12-26         193.050
      2023-12-27         193.150
      2023-12-28         193.580
      2023-12-29         192.530

[250 rows x 1 columns]

In [ ]:
!pip install "jupyterlab>=3" "ipywidgets>=7.6"
!pip install jupyter-dash
!pip install ta
!pip install plotly
!pip install pmdarima
!pip install nixtlats

#Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from scipy import stats
from prophet import Prophet
import plotly.express as px
import statsmodels.api as sm
from datetime import datetime
from nixtlats import TimeGPT
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels import regression
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

#Data Preprocessing

In [ ]:
data.to_csv('aapl_data.csv')

In [ ]:
data.head()

Sid               FIBBG000B9XRY4
Field Date                      
Close 2023-01-03         124.375
      2023-01-04         125.658
      2023-01-05         124.325
      2023-01-06         128.900
      2023-01-09         129.427

In [ ]:
data['FIBBG000B9XRY4']

Field  Date      
Close  2023-01-03    124.375
       2023-01-04    125.658
       2023-01-05    124.325
       2023-01-06    128.900
       2023-01-09    129.427
                      ...   
       2023-12-22    193.600
       2023-12-26    193.050
       2023-12-27    193.150
       2023-12-28    193.580
       2023-12-29    192.530
Name: FIBBG000B9XRY4, Length: 250, dtype: float64

In [ ]:
data = data.rename(columns = {'FIBBG000B9XRY4':'Close'})

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data

Sid                 Close
Field Date               
Close 2023-01-03  124.375
      2023-01-04  125.658
      2023-01-05  124.325
      2023-01-06  128.900
      2023-01-09  129.427
...                   ...
      2023-12-22  193.600
      2023-12-26  193.050
      2023-12-27  193.150
      2023-12-28  193.580
      2023-12-29  192.530

[250 rows x 1 columns]

In [ ]:
data.columns

Index(['Close'], dtype='object', name='Sid')

In [ ]:
data = data.reset_index()

In [ ]:
data

Sid,Field,Date,Close
0,Close,2023-01-03,124.375
1,Close,2023-01-04,125.658
2,Close,2023-01-05,124.325
3,Close,2023-01-06,128.900
4,Close,2023-01-09,129.427
...,...,...,...
245,Close,2023-12-22,193.600
246,Close,2023-12-26,193.050
247,Close,2023-12-27,193.150
248,Close,2023-12-28,193.580


In [ ]:
data.columns = ['Sid', 'Date', 'Close']

In [ ]:
data = data.drop('Sid',axis = 1)

In [ ]:
data

,Unnamed: 0,Date,Close
0,0,2023-01-03,124.375
1,1,2023-01-04,125.658
2,2,2023-01-05,124.325
3,3,2023-01-06,128.900
4,4,2023-01-09,129.427
...,...,...,...
245,245,2023-12-22,193.600
246,246,2023-12-26,193.050
247,247,2023-12-27,193.150
248,248,2023-12-28,193.580


In [4]:
from google.colab import files
upload = files.upload()

Saving aapl.csv to aapl (1).csv


In [5]:
data = pd.read_csv('aapl.csv')

In [6]:
data

,Unnamed: 0,Date,Close
0,0,2023-01-03,124.375
1,1,2023-01-04,125.658
2,2,2023-01-05,124.325
3,3,2023-01-06,128.900
4,4,2023-01-09,129.427
...,...,...,...
245,245,2023-12-22,193.600
246,246,2023-12-26,193.050
247,247,2023-12-27,193.150
248,248,2023-12-28,193.580


#Model

In [127]:
class Model():
  #Calculating Returns
  def Returns(self, prices):
    returns = [0]  # No return for the first day
    returns += [(prices[i] - prices[i-1]) / prices[i-1] for i in range(1, len(prices))]
    return returns

  #Classifying the states
  def states(self, returns):
    states = []
    for return_value in returns:
        if return_value >= 0.01:
            states.append(1)  # Bull state
        elif -0.01 < return_value < 0.01:
            states.append(0)  # Flat state
        else:
            states.append(-1)  # Bear state
    return np.array(states)

  #Calculating Transistion Distribution
  def transition_distribution(self, states):
    transition_matrix = np.zeros((3, 3))

    for i in range(1, len(states)):
        current_state = int(states[i-1] + 1)
        next_state = int(states[i] + 1)
        transition_matrix[current_state, next_state] += 1

    distribution = transition_matrix / np.sum(transition_matrix, axis=1, keepdims=True)

    return distribution

  def decide_to_buy(self, portfolio_values):
        if len(portfolio_values) < 2:
            return False  # Cannot make a decision if there's not enough data

        previous_value = portfolio_values[-2]
        current_value = portfolio_values[-1]

        # Buy if the current portfolio value is, for example, 10% higher than the previous value
        if current_value >= previous_value:
            return True
        else:
            return False

  #Maximizing the porfolio Value
  def Maximizing_value(self, price_history):
    portfolio = 0
    price_history = list(price_history)
    returns = self.Returns(price_history)
    states = self.states(returns)

    Buy_Indices = []
    portfolio_value = []
    date_index=[]
    date=[]

    for i in range(1, len(states)):
        if states[i] == 1 and states[i-1] == 0:
            portfolio += 1
            date_index.append(i)
            date.append(data.iloc[i]['Date'])
            portfolio_value.append(portfolio)
            Buy_Indices.append(i)

        elif states[i] == -1 and states[i-1] == 0:
            portfolio -= 1
            portfolio_value.append(portfolio)
        else:
            portfolio_value.append(portfolio)


    return portfolio_value, Buy_Indices, self.transition_distribution(states),states,date_index,date,returns

In [128]:
stock = Model()

Decide to Buy function leads to give that we can buy a stock or not for the particular day

In [132]:
Value = stock.Maximizing_value(data['Close'])

print("Maximizing portfolio:",Value[0])
decision = stock.decide_to_buy(Value[0])
if decision:
    print("Buy the stock for the day.")
else:
    print("Do not buy the stock for the day.")

Maximizing portfolio: [1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 8, 8, 7, 7, 7, 6, 6, 6, 5, 5, 6, 6, 6, 6, 5, 5, 5, 5, 5, 6, 6, 7, 7, 7, 7, 7, 6, 6, 7, 7, 8, 8, 8, 7, 7, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 18, 18, 18, 17, 17, 17, 17, 17, 17, 18, 18, 18, 17, 17, 16, 16, 16, 16, 16, 16, 16, 15, 15, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 15, 15, 15, 15, 14, 14, 14, 14, 15, 15, 14, 14, 14, 14, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 13, 13, 13, 12, 12, 12, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 19, 19, 18, 18, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18]
Buy the stock for the day.


In [130]:
labels = ['Bear', 'Flat', 'Bull']
fig = go.Figure(data=[go.Heatmap(z=Value[2], x=labels, y=labels, colorscale='electric')])
fig.update_layout(title='Transition Distribution Matrix', xaxis_title='Next State', yaxis_title='Current State')
fig.show()

In [131]:
import warnings
warnings.filterwarnings("ignore")

def get_day_of_week(date):
    try:
        # Get the day of the week as an integer (Monday=0, Sunday=6)
        day_of_week = date.weekday()
        # Convert the integer day of the week to a string representation
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        day_name = days[day_of_week]
        return day_name
    except ValueError:
        return "Invalid date format. Please provide date in YYYY-MM-DD format."

def counting_weekdays(start_date, end_date):
    if get_day_of_week(end_date) == 'Saturday' or get_day_of_week(end_date) == 'Sunday':
        print("It's " + get_day_of_week(end_date)+"  , we can't buy Stocks")
        return 0
    else:
        present_date = start_date + timedelta(days=1)  # Start counting from the day after the start date
        no_of_week_days = 0
        while present_date < end_date:  # Stop before the end date
            if present_date.weekday() < 5:  # Monday to Friday (0 to 4)
                no_of_week_days += 1
            present_date += timedelta(days=1)
        return no_of_week_days, get_day_of_week(end_date)

obj2 = Model()
val = obj2.Maximizing_value(data['Close'])
new_return_values = val[6]
print("new_Returns                      :", new_return_values)

given_date = datetime(2023, 12, 29)
last_date = datetime(2024, 1, 10)

no_of_week_days, day_of_week = counting_weekdays(given_date, last_date)
obj_new=Model()
values = obj_new.Maximizing_value(data['Close'])
return_values = values[6]

print("Maximized Porfolio Value     :", values[0][-1])
print("\n")
print("Transisition Distribution    :\n", values[2])
print("\n")
print("Buy_indices                  :\n", values[1])
print("\n")
print("Optimal Buy Dates            :", values[5])
print("\n")
print("Returns                      :", return_values)


new_Returns                      : [0, 0.010315577889447246, -0.010608158652851378, 0.036798713050472574, 0.004088440651667857, 0.004450385159201662, 0.021114897348522763, -0.0006026456142464225, 0.010123013839056812, 0.008753012812381087, -0.0053704580423718715, 0.00044623599934553743, 0.019224193044797073, 0.023500408449060418, 0.010062283539757466, -0.004705865752303557, 0.0148081462525962, 0.013683989941324415, -0.02008007221659457, 0.009022186280369797, 0.007903099910794036, 0.03706213439172453, 0.024396260884639434, -0.01792490285796103, 0.019239563381868496, -0.017647441316080363, -0.006917048373004233, 0.0024594917050476217, 0.01880319148936172, -0.004222465867857178, 0.013900813338489127, -0.010426497394991659, -0.007544630900979177, -0.02668263851409166, 0.002894238571814967, 0.0032904496048763356, -0.018004516250436853, 0.0082468159078, -0.003448229049293451, -0.01424251093219979, 0.004132031536106802, 0.03508784002312116, 0.018541417364712244, -0.014496537409682222, 0.00837

# Updated Transition Probability Matix


The make_trading_decision method in the Model_2 class calculates the transition probabilities based on the current state and determines the appropriate trading action. If the probability of a bullish transition is higher, it suggests a buying decision (1).

Conversely, if the probability of a bearish transition is higher, it recommends a selling decision (-1). In cases where probabilities are equal or no probabilities are associated with the current state, it advises holding (0). This method facilitates decision-making in trading scenarios based on the analysis of transition probabilities.






In [136]:
class Model_2:
    def __init__(self):
        self.value = 0
        self.state = 0  # Initial state: Flat
        self.transition_counts = {
            -1: {-1: 0, 0: 0, 1: 0},
            0: {-1: 0, 0: 0, 1: 0},
            1: {-1: 0, 0: 0, 1: 0}
        }

    #Calculating Returns
    def returns(self, current_price, prev_price):
        return (current_price - prev_price) / prev_price

    def portfolio_value(self, decision, prev_state, current_state):
      #print("prev_state :", prev_state, "current_state :", current_state)
      if(prev_state == 0 and current_state == 1):
        self.value += 1
      elif(prev_state == 0 and current_state == -1):
        self.value -= 1

    def updating_transition_counts(self, prev_state, current_state):
        self.transition_counts[prev_state][current_state] += 1


    #calculating Transition Probabilities
    def calculate_transition_probabilities(self):
          transition_probabilities = {}
          for prev_state, transitions in self.transition_counts.items():
              total_transitions = 0
              for value in transitions.values():  # Use a different variable name
                  total_transitions += value  # Accumulate the sum
              if total_transitions == 0:
                  continue
              transition_probabilities[prev_state] = {
                  next_state: count / total_transitions
                  for next_state, count in transitions.items()
              }
          return transition_probabilities

    def make_decision(self, current_price, prev_price):
        returns = self.returns(current_price, prev_price)

        if returns >= 0.01:
            current_state = 1  # Bull
        elif returns > -0.01 and returns < 0.01:
            current_state = 0  # Flat
        else:
            current_state = -1  # Bear


        state_matrix = {}

        prev_state = self.state

        # Calculate transition probabilities
        transition_probabilities = self.calculate_transition_probabilities()

        transistions = []
        # Decision making based on transition probabilities
        if self.state in transition_probabilities:
            prob_increase = transition_probabilities[self.state].get(1, 0)
            prob_decrease = transition_probabilities[self.state].get(-1, 0)

            if prob_increase > prob_decrease:
                decision = 1
                transistions.append('Bull')
                self.portfolio_value(decision, prev_state, current_state)
            elif prob_increase < prob_decrease:
                decision = 0
                transistions.append('Bear')
            else:
                decision = 0
                transistions.append('Flat')
        else:
            decision = 0

        #Updating Transistion Counts
        self.updating_transition_counts(self.state, current_state)
        self.state = current_state

        return decision, transistions, transition_probabilities

Values =  list(data['Close'])
model = Model_2()

for i in range(1, len(Values)):
    decision = model.make_decision(Values[i], Values[i-1])
    print(f"Day {i+1}: Decision: {decision}, Portfolio Value: {model.value}")

trans = model.make_decision(Values[i], Values[i-1])
print('\n')
print("Transistion Matrix   :",trans[2])
print('\n')
print("Current State        :",trans[1][-1])
print('\n')

transition_matrix = np.zeros((3, 3))
for prev_state, next_states in trans[2].items():
    for next_state, probability in next_states.items():
        transition_matrix[prev_state + 1][next_state + 1] = probability

print("Transition Matrix as 2D array:")
print(transition_matrix)
bull = transition_matrix[2]
print('\n')
print("Bull Row Values  :",bull)

#print("Transistion_Matrix                 :",transition_matrix)
# Multiply q by the forecasted_probabilities to get the probabilities after one time step
next_day_state_probabilities = np.dot(bull, transition_matrix)
print('\n')
states = ['Bear','Flat','Bull']
print("Probabilities of being in each state after one time step:",next_day_state_probabilities)
print('\n')

# Find the index of the maximum probability
max_probability_index = np.argmax(next_day_state_probabilities)
print("Index of the maximum probability   :", max_probability_index)
print('\n')
print("The Next day of the stock will be      :",states[max_probability_index])

Day 2: Decision: (0, [], {}), Portfolio Value: 0
Day 3: Decision: (0, [], {0: {-1: 0.0, 0: 0.0, 1: 1.0}}), Portfolio Value: 0
Day 4: Decision: (0, [], {0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 1.0, 0: 0.0, 1: 0.0}}), Portfolio Value: 0
Day 5: Decision: (0, ['Bear'], {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 1.0, 0: 0.0, 1: 0.0}}), Portfolio Value: 0
Day 6: Decision: (1, ['Bull'], {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}), Portfolio Value: 0
Day 7: Decision: (1, ['Bull'], {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.5, 1: 0.5}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}), Portfolio Value: 1
Day 8: Decision: (0, ['Bear'], {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.3333333333333333, 1: 0.6666666666666666}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}), Portfolio Value: 1
Day 9: Decision: (1, ['Bull'], {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.3333333333333333, 1: 0.6666666666666666}, 1: {-1: 0.3333333333333333, 0: 0.66666

Change of the Transistions Matrix in Every Iterations

In [137]:
class Portfolio:
    def __init__(self):
        self.value = 0
        self.state = 0  # Initial state: Flat
        self.transition_counts = {
            -1: {-1: 0, 0: 0, 1: 0},
            0: {-1: 0, 0: 0, 1: 0},
            1: {-1: 0, 0: 0, 1: 0}
        }

    def calculate_returns(self, current_price, prev_price):
        return (current_price - prev_price) / prev_price

    def update_portfolio_value(self, decision, prev_state, current_state):
      if(prev_state == 0 and current_state == 1):
        self.value += 1
      elif(prev_state == 0 and current_state == -1):
        self.value -= 1

    def update_transition_counts(self, prev_state, current_state):
        self.transition_counts[prev_state][current_state] += 1

    def calculate_transition_probabilities(self):
          transition_probabilities = {}
          for prev_state, transitions in self.transition_counts.items():
              total_transitions = 0
              for value in transitions.values():
                  total_transitions += value
              if total_transitions == 0:
                  continue
              transition_probabilities[prev_state] = {
                  next_state: count / total_transitions
                  for next_state, count in transitions.items()
              }
          return transition_probabilities


    def make_trading_decision(self, current_price, prev_price):
        returns = self.calculate_returns(current_price, prev_price)

        if returns >= 0.01:
            current_state = 1  # Bull
        elif returns > -0.01 and returns < 0.01:
            current_state = 0  # Flat
        else:
            current_state = -1  # Bear

        prev_state = self.state
        # Calculate transition probabilities
        transition_probabilities = self.calculate_transition_probabilities()
        print("Matrix: ", transition_probabilities)
        decision = 0
        transistions = []
        # Decision making based on transition probabilities
        if self.state in transition_probabilities:
            prob_increase = transition_probabilities[self.state].get(1, 0)
            prob_decrease = transition_probabilities[self.state].get(-1, 0)
            #print("Increase: ", prob_increase, " Decrease: ",prob_decrease)
            if prob_increase > prob_decrease:
                transistions.append('Bull')
                decision = 1
                self.update_portfolio_value(decision, prev_state, current_state)
            elif prob_increase < prob_decrease:
                decision = 0
                transistions.append('Bear')
            else:
                decision = 0
                transistions.append('Flat')
        else:
            decision = 0
        # Update transition counts and probabilities
        self.update_transition_counts(self.state, current_state)
        #print("Update_transition_counts: ",self.transition_counts)
        self.state = current_state

        #print("Decision :",decision)

        return decision, transistions, transition_probabilities


prices =  list(data['Close'])
portfolio = Portfolio()

for i in range(1, len(prices)):
    decision = portfolio.make_trading_decision(prices[i], prices[i-1])[0]
    print(f"Day {i+1}: Decision: {decision}, Portfolio Value: {portfolio.value}")

trans = portfolio.make_trading_decision(prices[i], prices[i-1])
print('\n')
print("Current State        :",trans[1][-1])
print('\n')
print("Transistion Matrix   :",trans[2])
print('\n')

transition_matrix = np.zeros((3, 3))
for prev_state, next_states in trans[2].items():
    for next_state, probability in next_states.items():
        transition_matrix[prev_state + 1][next_state + 1] = probability

print("Transition Matrix as 2D array:")
print(transition_matrix)
bull = transition_matrix[2]
print(bull)

#print("Transistion_Matrix                 :",transition_matrix)
# Multiply q by the forecasted_probabilities to get the probabilities after one time step
next_day_state_probabilities = np.dot(bull, transition_matrix)
print('\n')
states = ['Bear','Flat','Bull']
print("Probabilities of being in each state after one time step:",next_day_state_probabilities)
print('\n')

# Find the index of the maximum probability
max_probability_index = np.argmax(next_day_state_probabilities)
print("Index of the maximum probability   :", max_probability_index)
print('\n')
print("That day of the stock will be      :",states[max_probability_index])

Matrix:  {}
Day 2: Decision: 0, Portfolio Value: 0
Matrix:  {0: {-1: 0.0, 0: 0.0, 1: 1.0}}
Day 3: Decision: 0, Portfolio Value: 0
Matrix:  {0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 1.0, 0: 0.0, 1: 0.0}}
Day 4: Decision: 0, Portfolio Value: 0
Matrix:  {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 1.0, 0: 0.0, 1: 0.0}}
Day 5: Decision: 0, Portfolio Value: 0
Matrix:  {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.0, 1: 1.0}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}
Day 6: Decision: 1, Portfolio Value: 0
Matrix:  {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.5, 1: 0.5}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}
Day 7: Decision: 1, Portfolio Value: 1
Matrix:  {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.3333333333333333, 1: 0.6666666666666666}, 1: {-1: 0.5, 0: 0.5, 1: 0.0}}
Day 8: Decision: 0, Portfolio Value: 1
Matrix:  {-1: {-1: 0.0, 0: 0.0, 1: 1.0}, 0: {-1: 0.0, 0: 0.3333333333333333, 1: 0.6666666666666666}, 1: {-1: 0.3333333333333333, 0: 0.6666666666666666, 1: 0.0}}
Day 9: Dec